# 


# CuPy Lab 2:  Serial Code Lab Assignment
---

#### [<<--CuPy Lab 1](cupy_guide.ipynb)


## A Recap on RDF

- The radial distribution function (RDF) denoted in equations by g(r) defines the probability of finding a particle at a distance r from another tagged particle. 
- The RDF is strongly dependent on the type of matter so will vary greatly for solids, gases and liquids.
- It is observed the code complexity of the algorithm in 𝑁^2. Let us get into details of the accelerated code analysis. 
- The input data for the serial code is fetched from a DCD binary trajectory file.


### The Serial Code
- The Cell below consist of two functions namely **dcdreadhead** and **dcdreadframe**
- The **dcdreadhead** function computes the total number of frames and atoms from the DCDFile **(input/alk.traj.dcd)**, while the **dcdreadframe** function reads 10 frames and 6720 atoms (note: each frame contains 6720 atoms) using the MDAnalysis library. 
- Both functions run on the Host (CPU) and are being called from the function **main()**.
### <u>Cell 1</u>

In [ ]:
import cupy as cp
import numpy as np
import math
import cupy.cuda.nvtx as nvtx
from MDAnalysis.lib.formats.libdcd import DCDFile
from timeit import default_timer as timer


def dcdreadhead(infile):
    nconf   = infile.n_frames
    _infile = infile.header
    numatm  = _infile['natoms']
    return numatm, nconf

def dcdreadframe(infile, numatm, nconf):

    d_x = np.zeros(numatm * nconf, dtype=np.float64)
    d_y = np.zeros(numatm * nconf, dtype=np.float64)
    d_z = np.zeros(numatm * nconf, dtype=np.float64)

    for i in range(nconf):
        data = infile.readframes(i, i+1)
        box = data[1]
        atomset = data[0][0]
        xbox = round(box[0][0], 8)
        ybox = round(box[0][2],8)
        zbox = round(box[0][5], 8)

        for row in range(numatm):
            d_x[i * numatm + row] = round(atomset[row][0], 8) # 0 is column
            d_y[i * numatm + row] = round(atomset[row][1], 8)  # 1 is column
            d_z[i * numatm + row] = round(atomset[row][2], 8)  # 2 is column

    return xbox, ybox, zbox, d_x, d_y, d_z


##  pair_gpu function

- The pair_gpu is the function where the main task of the RDF serial implementation is being executed. The function computes differences in xyz DCD frames.
- The essence of njit(just-in-time) decorator is to get pair_gpu function to compile under no python mode, and this is really important for good performance. 
- The decorator **@njit** or **@jit(nopython=True)** ensures that an exception is raised when compilation fails as a way of to alert that a bug is found within the decorated function. You can read more [here](https://numba.pydata.org/numba-doc/latest/user/performance-tips.html).

### <u>Cell 2</u>

In [ ]:
from numba import njit

@njit()
def pair_gpu(d_x, d_y, d_z, d_g2, numatm, nconf, xbox, ybox, zbox, d_bin):
    box = min(xbox, ybox)
    box = min(box, zbox)
    _del = box / (2.0 * d_bin)
    cut = box * 0.5

    for frame in range(nconf):
       # print("\n {}".format(frame))
        for id1 in range(numatm):
            for id2 in range(numatm):
                dx = d_x[frame * numatm + id1] - d_x[frame * numatm + id2]
                dy = d_y[frame * numatm + id1] - d_y[frame * numatm + id2]
                dz = d_z[frame * numatm + id1] - d_z[frame * numatm + id2 ]
                dx = dx - xbox * (round(dx / xbox))
                dy = dy - ybox * (round(dy / ybox))
                dz = dz - zbox * (round(dz / zbox))

                r = math.sqrt(dx * dx + dy * dy + dz * dz)
                if r < cut :
                    ig2  = int((r/_del))
                    d_g2[ig2] = d_g2[ig2] + 1

    return d_g2

#### Brief Analysis on internal task performed within pair_gpu function
- The graphic below identifies the various operations executed in the pair_gpu function. This function executes three nested-loops using tricky indexing manipulation within the arrays.


<img src="../images/pair_gpu.png" width="80%"/>

- The indexing flow for the operation 1 is simulated using the graphic below. Each green box simualate the subtraction operation within the two inner loops (id1 & id2) while the indexes written in blue signifies the outer-most loop (frame) which iterates 10 times. 

<img src="../images/pair_gpu_analysis.png" width="80%"/>





### The Main Function
- This is the entry point of the program where every other functions including the **pair_gpu** function are called. The output of the main function is written into two files. An image version of the output files ("**cupy_RDF.dat**" & "**cupy_Pair_entropy.dat**") are displayed below the code cell.

### <u>Cell 3</u>

In [ ]:
from MDAnalysis.lib.formats.libdcd import DCDFile
import os
from pathlib import Path

def main():
    start = timer()
    ########## Input Details ###########
    inconf = 10
    nbin   = 2000
    global xbox, ybox, zbox
    
    fileDir = os.path.dirname(os.path.realpath('__file__'))
    dataRoot = Path(fileDir).parents[1]
    file = os.path.join(dataRoot, 'source_code/input/alk.traj.dcd')
    
    infile = DCDFile(file)
    pairfile = open("RDF.dat", "w+")
    stwo     = open("Pair_entropy.dat", "w+")

    numatm, nconf = dcdreadhead(infile)
    print("Dcd file has {} atoms and {} frames".format(numatm, nconf))
    if inconf > nconf:
        print("nconf is reset to {}".format(nconf))
    else:
        nconf = inconf
    print("Calculating RDF for {} frames".format(nconf))
    #numatm = 50
    sizef =  nconf * numatm
    sizebin = nbin
    ########### reading cordinates ##############
    nvtx.RangePush("Read_File")
    xbox, ybox, zbox, h_x, h_y, h_z = dcdreadframe(infile, numatm, nconf)
    nvtx.RangePop() # pop for reading file

    h_g2 = np.zeros(sizebin, dtype=np.longlong)
    print("Reading of input file is completed")
   
    print("\n {} {}".format(nconf, numatm))
    ############# This where we will concentrate #########################
    nvtx.RangePush("Pair_Circulation")
    h_g2 = pair_gpu(h_x, h_y, h_z, h_g2, numatm, nconf, xbox, ybox, zbox, nbin)
    nvtx.RangePop() #pop for Pair Calculation
    ######################################################################
    
    pi = math.acos(np.int64(-1.0))
    rho = (numatm) / (xbox * ybox * zbox)
    norm = (np.int64(4.0) * pi * rho) / np.int64(3.0)
    g2 = np.zeros(nbin, dtype=np.float32)
    s2 = np.int64(0.0);
    s2bond = np.int64(0.0)
    lngrbond = np.int64(0.0)
    box = min(xbox, ybox)
    box = min(box, zbox)
    _del = box / (np.int64(2.0) * nbin)
    gr = np.float32(0.0)
    # loop to calculate entropy
    nvtx.RangePush("Entropy_Calculation")
    for i in range(nbin):
        rl = (i) * _del
        ru = rl + _del
        nideal = norm * (ru * ru * ru - rl * rl * rl)
        g2[i] = h_g2[i] / (nconf * numatm * nideal)
        r = (i) * _del
        temp = (i + 0.5) * _del
        
        #writing to file
        pairfile.write(str(temp) + " " + str(g2[i]) + "\n")

        if r < np.int64(2.0):
            gr = np.int64(0.0)
        else:
            gr = g2[i]
        if gr < 1e-5:
            lngr = np.int64(0.0)
        else:
            lngr = math.log(gr)
        if g2[i] < 1e-6:
            lngrbond = np.int64(0.0)
        else:
            lngrbond = math.log(g2[i])
        s2 = s2 - (np.int64(2.0) * pi * rho * ((gr * lngr) - gr + np.int64(1.0)) * _del * r * r)
        s2bond = s2bond - np.int64(2.0) * pi * rho * ((g2[i] * lngrbond) - g2[i] + np.int64(1.0)) * _del * r * r

    nvtx.RangePop() # pop for entropy Calculation
    
    #writing s2 and s2bond to file
    stwo.writelines("s2 value is {}\n".format(s2))
    stwo.writelines("s2bond value is {}".format(s2bond))
    
    # printing s2 and s2bond to jupyter output
    print("\n s2 value is {}\n".format(s2))
    print("s2bond value is {}\n".format(s2bond))

    print("#Freeing Host memory")
    del(h_x)
    del(h_y)
    del(h_z)
    del(h_g2)
    print("#Number of atoms processed: {}  \n".format(numatm))
    print("#number of confs processed: {} \n".format(nconf))
    

if __name__ == "__main__":
    #main()  

---

###  Output Files
<table>
    <tr>
    <td>
         <img src="../images/serial_output_file.png" width="95%" />
    </td>
    </tr>
</table>


---

# Lab Task 

1. **Run the serial code from cell 1, 2, & 3**
    - Remove the **"#"** behind the **main()** before running the cell 3:
    ```python
       if __name__ == "__main__":
                main()
    ```
2. **Now, lets start modifying the original code to CuPy code constructs.**
> From the top menu, click on File, and Open **nways_serial.py** from the current directory at **Python/source_code/cupy** directory. Remember to SAVE your code after changes, before running below cells. 
> Hints: focus on the **pair_gpu** function and you may as well need to modify few lines in the **main** function.

In [ ]:
%run ../../source_code/serial/nways_serial.py

The output should be the following:

```
s2 value is -2.43191
s2bond value is -3.87014
```

3. **Profile the code by running the cell bellow** 

In [ ]:
!cd ../../source_code/serial&& nsys profile --stats=true --force-overwrite true -o serial_cpu_rdf python3 nways_serial.py

To view the profiler report, you would need to [Download the profiler output](../../source_code/serial/serial_cpu_rdf.qdrep) and open it via the GUI. A sample expected profile report should is shown below.

<img src="../images/cupy_nsys1.png"/>
<img src="../images/cupy_nsys3.png"/>

From the profile report, we can see that the pair_gpu function now takes miliseconds to run as compared to the serial version which takes more than 3 seconds as shown [here](../serial/rdf_overview.ipynb). 
 

---
### [View ](../../source_code/cupy/cupy_rdf.py) or [Run](../../jupyter_notebook/cupy/cupy_RDF.ipynb) Solution
---



## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well. You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.



In [ ]:
%%bash
cd ..
rm -f nways_files.zip
zip -r nways_files.zip *

**After** executing the above zip command, you should be able to download the zip file [here](../nways_files.zip).

**IMPORTANT**: Please click on **HOME** to go back to the main notebook for *N ways of GPU programming for MD* code.

---

# <p style="text-align:center;border:3px; border-style:solid; border-color:#FF0000  ; padding: 1em"> <a href=../../../nways_MD_start_python.ipynb>HOME</a></p>

---


## Links and Resources

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[CUDA Toolkit Download](https://developer.nvidia.com/cuda-downloads)

**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 


## Licensing 

This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0).